In [1]:
import pandas as pd
from pathlib import Path


In [2]:
jan_data_path = Path("Data/JC-202001-citibike-tripdata.csv")
feb_data_path = Path("Data/JC-202002-citibike-tripdata.csv")
mar_data_path = Path("Data/JC-202003-citibike-tripdata.csv")
apr_data_path = Path("Data/JC-202004-citibike-tripdata.csv")
may_data_path = Path("Data/JC-202005-citibike-tripdata.csv")

jan_data = pd.read_csv(jan_data_path)
feb_data = pd.read_csv(feb_data_path)
mar_data = pd.read_csv(mar_data_path)
apr_data = pd.read_csv(apr_data_path)
may_data = pd.read_csv(may_data_path)

In [3]:
print(f'January 2020: {jan_data.shape}')
print(f'February 2020: {feb_data.shape}')
print(f'March 2020: {mar_data.shape}')
print(f'April 2020: {apr_data.shape}')
print(f'May 2020: {may_data.shape}')

January 2020: (26020, 15)
February 2020: (22962, 15)
March 2020: (17719, 15)
April 2020: (9268, 15)
May 2020: (25077, 15)


In [4]:
# create subsets of csv data
numOfLines = 50
divby = 1000
jan_subset = pd.read_csv(jan_data_path, nrows=(len(jan_data)//divby))
feb_subset = pd.read_csv(feb_data_path, nrows=(len(feb_data)//divby))
mar_subset = pd.read_csv(mar_data_path, nrows=(len(mar_data)//divby))
apr_subset = pd.read_csv(apr_data_path, nrows=(len(apr_data)//divby))
may_subset = pd.read_csv(may_data_path, nrows=(len(mar_data)//divby))


In [5]:
# insert Month column
jan_subset.insert(0, "Month", "Jan")
feb_subset.insert(0, "Month", "Feb")
mar_subset.insert(0, "Month", "Mar")
apr_subset.insert(0, "Month", "Apr")
may_subset.insert(0, "Month", "May")


In [6]:
# # create combined csv file
# # combined_data = combined_data.append(feb_subset, ignore_index=True)
# # combined_data = combined_data.append(mar_subset, ignore_index=True)
# # combined_data = combined_data.append(apr_subset, ignore_index=True)

# dfs = []
# dfs.append(feb_subset)
# dfs.append(mar_subset)
# dfs.append(apr_subset)
# dfs.append(may_subset)

# df_combined = pd.concat(dfs, ignore_index=True)
# df_combined.to_csv("combined_data.csv", index=False)

# # create subsets from each csv file
# jan_subset.to_csv("jan_subset.csv")
# feb_subset.to_csv("feb_subset.csv")
# mar_subset.to_csv("mar_subset.csv")
# apr_subset.to_csv("apr_subset.csv")
# may_subset.to_csv("may_subset.csv")

In [7]:
# # create groupby start station name with number of rows for each station
# grp = apr_data.groupby("start station name").size()

# # df_trips = grp.map("{:,}".format).to_frame("Trips")
# grp.to_csv("apr_startstations.csv")
# # df_trips = grp.to_frame("Trips").sort_values(["Trips"], ascending=False)
# # df_trips.head(None)

In [8]:
# # jan_data.sort_values("start station name", inplace=False)
# # janfilter = jan_data["start station name"]=="Astor Place"
# # janfilter.count()
# # jancount = jan_data[(jan_data["start station name"]=="Astor Place")]
# # groupby start station name with number of rows for each station
# jan_grouped = jan_data.groupby("start station name").count()
# jan_grouped = jan_grouped.sort_values("tripduration", ascending=False).reset_index()
# # include only station name and number of rows
# jan_grouped = jan_grouped[["start station name", "tripduration"]]
# jan_grouped.head(10)


In [9]:
# # get number of unique stations
# start_ids = jan_data.groupby(["start station name"]).count()
# end_ids = jan_data.groupby(["end station name"])
# print(len(start_ids))
# print(len(end_ids))
# start_ids.head()
